In [146]:
#library imports.
from bs4 import BeautifulSoup
import requests
import csv

#this fucntion takes in rows of information and return a .csv in the desired format.
def build_csv(rows: list, file_name: str):
    content = []
    for i in range(len(rows[0])):
        current_row = []
        for j in range(len(rows)):
            current_row.append(rows[j][i])
        content.append(current_row)
    with open(file_name, 'w', newline='') as csvfile: 
        csv_writer = csv.writer(csvfile)
        for row in content: 
            csv_writer.writerow(row)  

#this function takes in the html file that contains course information and scrapes required data into appropriate output.
def search_NEU(html_file_name: str): 
    #calling in the downloaded .html file.
    with open(html_file_name, "r") as f:
        doc = BeautifulSoup(f, "html.parser")

    #scraping course codes and names.
    courses = doc.find_all('span', class_ = "SearchResult__header--classTitle")
    names_codes = []
    for course in courses:
        names_codes.append(course.get_text())
    codes = []
    names = []
    for chars in names_codes:
        parts = chars.split(':')
        codes.append(parts[0])
        names.append(parts[1][1:])

    #scraping course requirements.
    conditions = doc.find_all('div', class_ = "SearchResult__panel--left")
    reqs = []
    for condition in conditions:
        reqs.append(condition.get_text())
    paths = []
    pre = []
    coreq = []
    for chars in reqs:
        parts = chars.split(':')
        chars = parts[1]
        paths.append(chars[:chars.index("Prerequisites")])
        chars = parts[2]
        pre.append(chars[:chars.index("Corequisites")])
        chars = parts[3]
        coreq.append(chars[:chars.index("Course fees")])

    #scraping credits.
    credits = doc.find_all('span', class_ = "SearchResult__header--creditString")
    points=[]
    for credit in credits:
        points.append(credit.get_text())
    cre = []
    for chars in points:
        parts = chars.split(' ')
        cre.append(parts[0])

    #scraping course descriptions.   
    descs = doc.find_all('div', class_ = "SearchResult__panel")
    texts = []
    for desc in descs:
        first_span = desc.find('span')
        text = first_span.text.strip()
        texts.append(text)
    
    #intializing a main list.
    main = []
    main = [codes, names, paths, pre, coreq, cre, texts]
    return(main)